# Layer 5 of the microcircuit model

This example simulates the excitatory and inhibitory neuron populations of layer 5 of the cortical microcircuit model by Potjans & Diesmann (2014):

[1] Potjans, T. C., & Diesmann, M. (2014). The cell-type specific cortical microcircuit: relating structure and activity in a full-scale spiking network model. Cerebral Cortex 24(3):785-806

Contact: Johanna Senk (j.senk@fz-juelich.de)

First, import NEST and all necessary modules for simulation, analysis and plotting.

In [ ]:
%pylab inline
import nest
import numpy as np
import nest.raster_plot

## Parameters

The following parameters are extracted from or computed based on Table 5 of Ref. [1].

In [ ]:
# simulation parameters
T = 1000.                  # simulation time (ms)
dt = 0.1                   # simulation resolution (ms)

# network parameters
N_L5E = 4850               # number of neurons in L5E
N_L5I = 1065               # number of neurons in L5I
    
Nsyn_L5E_L5E = 2038173     # number of synapses with presynaptic neuron (pre) in L5E and postsynaptic neuron (post) in L5E 
Nsyn_L5E_L5I = 319602      # number of synapses with pre in L5E and post in L5I
Nsyn_L5I_L5I = 430775      # number of synapses with pre in L5I and post in L5I
Nsyn_L5I_L5E = 2411184     # number of synapses with pre in L5I and post in L5E

K_L5E_ext = 2000           # indegree of excitatory neurons from external poisson drive
K_L5I_ext = 1900           # indegree of inhibitory neurons from external poisson drive

# neuron parameters
neuron_params = {
    'C_m'       : 250.,    # membrane capacity (pF)
    'I_e'       : 0.0,     # external input current (pA)
    'tau_m'     : 10.0,    # membrane time constant (ms)
    't_ref'     : 2.0,     # absolute refractory period (ms)
    'tau_syn_ex': 0.5,     # excitatory postsynaptic current time constant (ms)
    'tau_syn_in': 0.5,     # inhibitory postsynaptic current time constant (ms)
    'V_reset'   : -65.0,   # reset potential (mV)
    'E_L'       : -65.0,   # resting potential (mV)
    'V_th'      : -50.0}   # spike threshold (mV)

# synapse parameters
w = 87.8                   # mean excitatory weight (pA)
sigma_w = 8.8              # standard deviation of excitatory weight (pA)
g = -4.                    # relative inhibitory weight 

de = 1.5                   # mean spike transmission delay for excitatory presynaptic neurons (ms)
sigma_de = 0.75            # standard deviation 
di = 0.8                   # mean spike transmission delay for inhibitory presynaptic neurons (ms)
sigma_di = 0.4             # standard deviation
 
# input parameters
bg_rate = 8.               # external Poisson rate (spikes/s)
perturbation = False

# 1) Create and connect neurons

Reset the simulation kernel for avoiding interferences with previous NEST simulations.

In [ ]:
nest.ResetKernel()

In [ ]:
nest.SetKernelStatus({'resolution': dt})      # set simulation resolution

Create the excitatory and inhibitory neuron populations with neurons of type 'iaf_psc_exp', the correct population sizes, and the given neuron parameters.
The neuron parameters are set in two alternative ways.

In [ ]:
pop_L5E = nest.Create('iaf_psc_exp', N_L5E, params=neuron_params)

pop_L5I = nest.Create('iaf_psc_exp', N_L5I)
pop_L5I.set(neuron_params)

The recurrent connections are now established. Weights and delays are drawn from normal distributions.
We begin with the excitatory connections (from L5E).

In [ ]:
syn_dict_E = {
    'synapse_model': 'static_synapse',
    'weight': nest.math.redraw(
        nest.random.normal(
            mean=w,
            std=sigma_w),
        min=0.,
        max=np.Inf),
    'delay': nest.math.redraw(
        nest.random.normal(
            mean=de,
            std=sigma_de),
        min=dt,
        max=np.Inf)}

# connections to L5E
# specifying the connection parameters
conn_dict_EE = {'rule': 'fixed_total_number', 'N': Nsyn_L5E_L5E}
nest.Connect(pop_L5E, pop_L5E, conn_dict_EE, syn_dict_E)

# connections to L5I
conn_dict_EI = {'rule': 'fixed_total_number', 'N': Nsyn_L5E_L5I}
nest.Connect(pop_L5E, pop_L5I, conn_dict_EI, syn_dict_E)

Next, establish the inhibitory connections (from L5I).

In [ ]:
syn_dict_I = {
    'synapse_model': 'static_synapse',
    'weight': nest.math.redraw(
        nest.random.normal(
            mean=g*w,
            std=np.abs(g*sigma_w)),
        min=np.NINF,
        max=0.),
    'delay': nest.math.redraw(
        nest.random.normal(
            mean=di,
            std=sigma_di),
        min=dt,
        max=np.Inf)}

# connections to L5E
# specifying the connection parameters
conn_dict_IE = {'rule': 'fixed_total_number', 'N': Nsyn_L5I_L5E}
nest.Connect(pop_L5I, pop_L5E, conn_dict_IE, syn_dict_I)

# connections to L5I
conn_dict_II = {'rule': 'fixed_total_number', 'N': Nsyn_L5I_L5I}
nest.Connect(pop_L5I, pop_L5I, conn_dict_II, syn_dict_I)

# 2) Create and connect devices

Poisson generators simulate neuron firing with the statistics of Poisson processes. Here, they emulate external excitatory input to the network.

Create two Poisson generators and connect them to the respective populations.
The given external rate 'bg_rate' corresponds to the rate communicated by one synapse.

In [ ]:
# connections to L5E
poisson_generator_L5E = nest.Create('poisson_generator', params={'rate': bg_rate * K_L5E_ext})
nest.Connect(poisson_generator_L5E, pop_L5E, 'all_to_all', {'weight': w, 'delay': de})

# connections to L5I
poisson_generator_L5I = nest.Create('poisson_generator', params={'rate': bg_rate * K_L5I_ext})
nest.Connect(poisson_generator_L5I, pop_L5I, 'all_to_all', {'weight': w, 'delay': de})

Set up and connect two spike detectors, one for each population.

In [ ]:
sd_L5E = nest.Create('spike_detector')
nest.Connect(pop_L5E, sd_L5E, 'all_to_all')

sd_L5I = nest.Create('spike_detector')
nest.Connect(pop_L5I, sd_L5I, 'all_to_all')

# 4*) Perturbation

If 'perturbation=True' in the section with the parameter definitions above, the network shall experience a perturbation for a certain time interval during the simulation. In our case, the perturbation consists of an additional Poisson generator which connects to 'N_pert' excitatory parrot neurons connecting to L5E. Parrot neurons just repeat incoming spikes and, hence, they can be used to generate correlated input.

In [ ]:
if perturbation == True:
    
    # parameters
    N_pert = 1000
    Nsyn_L5E_pert = 500000
    start = 100.
    stop = 300.
    pert_rate = 8.
    
    poisson_generator_stimulus = nest.Create('poisson_generator',
                                             params={'rate': pert_rate,
                                                     'start': start,
                                                     'stop': stop})
    parrot_neurons = nest.Create('parrot_neuron', N_pert)
    nest.Connect(poisson_generator_stimulus, parrot_neurons, 'all_to_all')
    
    conn_dict = {'rule': 'fixed_total_number', 'N': Nsyn_L5E_pert}
    nest.Connect(parrot_neurons, pop_L5E, conn_dict, syn_dict_E) 
    

# 3) Run simulation and analysis

Run the simulation!

In [ ]:
nest.Simulate(T)

After the simulation, the recorded spikes are read out. We can extract the spike time and the neuron ID of the sending neuron from each event.

In [ ]:
spike_senders_L5E = nest.GetStatus(sd_L5E)[0]['events']['senders']
spike_times_L5E = nest.GetStatus(sd_L5E)[0]['events']['times']
spike_senders_L5I = nest.GetStatus(sd_L5I)[0]['events']['senders']
spike_times_L5I = nest.GetStatus(sd_L5I)[0]['events']['times']

# stack the data from the two populations
spike_senders = np.hstack((spike_senders_L5E, spike_senders_L5I))
spike_times = np.hstack((spike_times_L5E, spike_times_L5I))

Finally, we create a raster plot to visualize the spiking activity of all neurons during the simulation ...

In [ ]:
pylab.plot(spike_times, spike_senders, 'k.', markersize=1)
pylab.xlim(0,T)
pylab.ylim(pop_L5E[0].global_id, pop_L5I[-1].global_id)
pylab.xlabel('time (ms)')
pylab.ylabel('neuron ID')

... and we compute the average firing rate for each population.

Therefore, we read out the number of spikes registered by a spike detector during the simulation time. Pay attention to get the correct time units.

In [ ]:
rate_E = float(nest.GetStatus(sd_L5E)[0]['n_events']) / T * 1e3 / N_L5E
print("\nFiring rate E = %.1f spikes/s" % (rate_E))

rate_I = float(nest.GetStatus(sd_L5I)[0]['n_events']) / T * 1e3 / N_L5I
print("\nFiring rate I = %.1f spikes/s" % (rate_I))

Besides, NEST also provides built-in plotting tools for creating raster plots.

In [ ]:
nest.raster_plot.from_device(sd_L5E, hist=True)
nest.raster_plot.from_device(sd_L5I, hist=True)